# Tools

This notebook contains all the tools that will be used by the agent.

In [1]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location

The following function look up at latitude and longitude of a city.

In [2]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master
df = pd.read_csv('/content/data/cities_latlng.csv', sep=';')


In [3]:
# TODO: Explore the loaded dataframe
print(df.head())


   id      country            city   latitude  longitude  altitude
0   1  Afghanistan           Kabul  34.516667  69.183334      1808
1   2  Afghanistan        Kandahar  31.610000  65.699997      1015
2   3  Afghanistan  Mazar-e Sharif  36.706944  67.112221       369
3   4  Afghanistan           Herat  34.340000  62.189999       927
4   5  Afghanistan       Jalalabad  34.420000  70.449997       573


In [16]:
# TODO: Add tool description

@tool
def get_latlng(city: str) -> any:
   """
   Return the latitue, longitude and altitude of a city

   Args:
     city: the name of the city that we want to get the latitue, longitude and altitude

   Returns:
     any: A dictionary with the following keys: city, latitude, longitude, altitude

   Example:
     get_latlng('Tokyo')

   """
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   return { 'city': city, 'latitude': r.iloc[0]['latitude'], 'longitude': r.iloc[0]['longitude'], 'altitude': r.iloc[0]['altitude'] }





In [7]:
# TODO: Test get_latlng method
# case insensitive search
get_latlng('tokyo')


{'city': 'tokyo',
 'latitude': np.float64(35.6895266),
 'longitude': np.float64(139.6916809),
 'altitude': np.int64(40)}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [18]:
# TODO: Add tool description
@tool
def get_temperature(latitude: float, longitude: float) -> any:
   """
   Returns the temperature of a location given by its latitude and longitude

   Args:
     latitude: the latitude of the location
     longitude: the longitude of the location

   Returns:
     any: a dictionary with the following keys: temperature_unit, temperature

   Example:
     get_temperature(35.6895266, 139.6916809)
   """
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }



In [19]:
# TODO: Test get_temperature method
get_temperature(35.6895266, 139.6916809)

{'temperature_unit': '°C', 'temperature': 7.6}

### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------|
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [23]:
# TODO: Add tool description
@tool
def query_album_track(query: str) -> any:
   """
   Perform a query on the `album_track` table. Returns the result as an array of tuples
   The table album_track has the following columns and data types:
    AlbumId: integer
    Title: nvarchar(160)
    track_name: nvarchar(200)
    artist_name: nvarchar(120)
    duration: integer
    composer: nvarchar(220)
   The duration is in milliseconds

   Args:
     query: a valid SQL query

   Returns:
     any: list of tuples with the result of the query

   Example:
    result = album_track.query("select * from album_track limit 10")

   """
   database = "/content/data/chinook_sqlite.sqlite"
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor()
      rows = cursor.execute(query)
      return rows.fetchall()
   finally:
      conn.close()

In [24]:
# TODO: Test the query_album_track function
query_album_track("select * from album_track limit 10")

[(1,
  'For Those About To Rock We Salute You',
  'For Those About To Rock (We Salute You)',
  'AC/DC',
  343719,
  'Angus Young, Malcolm Young, Brian Johnson'),
 (2, 'Balls to the Wall', 'Balls to the Wall', 'Accept', 342562, None),
 (3,
  'Restless and Wild',
  'Fast As a Shark',
  'Accept',
  230619,
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 (3,
  'Restless and Wild',
  'Restless and Wild',
  'Accept',
  252051,
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 (3,
  'Restless and Wild',
  'Princess of the Dawn',
  'Accept',
  375418,
  'Deaffy & R.A. Smith-Diesel'),
 (1,
  'For Those About To Rock We Salute You',
  'Put The Finger On You',
  'AC/DC',
  205662,
  'Angus Young, Malcolm Young, Brian Johnson'),
 (1,
  'For Those About To Rock We Salute You',
  "Let's Get It Up",
  'AC/DC',
  233926,
  'Angus Young, Malcolm Young, Brian Johnson'),
 (1,
  'For Those About To Rock We Salute You',
  'Inject The Venom',
  'AC/DC',
  210834,
  'An

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [25]:
class SQLiteTool(Tool):

   # tools name
   name = "album_query"

   description = """
   Perform a query on the `album_track` table. Returns the result as an array of tuples
   The table album_track has the following columns and data types:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name: nvarchar(120)
      duration: integer
      composer: nvarchar(220)
   The duration is in milliseconds
   """

   # parameters for forward()
   inputs = {
     "query": {
         "type": "string",
         "description": "A valid SQL query"
     }
   }

   # output
   output_type = "any"
   output_description = "list of tuples with the result of the query"

   def __init__(self, db_file):
      self.db_file = db_file
      self.setup()

   def setup(self):
      super().setup()
      self.conn = sqlite3.connect(self.db_file)

   def forward(self, query: str) -> any:
      try:
         cursor = self.conn.cursor()
         rows = cursor.execute(query)
         return rows.fetchall()
      except Exception as e:
         print(f'Query exception: {e}')